In [3]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 17.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which

In [4]:
import os
import re
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification
from datasets import Dataset, DatasetDict

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
data_dir = '/content/drive/MyDrive/Fake_new_data_2'
files = ['train.tsv', 'valid.tsv', 'test.tsv']
dfs = [pd.read_csv(os.path.join(data_dir, f), sep='\t', header=None) for f in files]

In [7]:
cols = ['id', 'label', 'statement', 'subjects', 'speaker', 'job_title', 'state_info',
        'party_affiliation', 'barely_true_cnt', 'false_cnt', 'half_true_cnt',
        'mostly_true_cnt', 'pants_on_fire_cnt', 'context']
for df in dfs:
    df.columns = cols
train_df, val_df, test_df = dfs[0], dfs[1], dfs[2]

In [8]:
encoder = LabelEncoder()
train_df['label_enc'] = encoder.fit_transform(train_df['label'])
val_df['label_enc'] = encoder.transform(val_df['label'])
test_df['label_enc'] = encoder.transform(test_df['label'])
num_labels = len(encoder.classes_)

In [9]:
def clean_text(s):
    s = s.lower()
    s = re.sub(r"[^a-z0-9 ]", " ", s)
    return s.strip()

train_df['clean'] = train_df['statement'].apply(clean_text)
val_df['clean']   = val_df['statement'].apply(clean_text)
test_df['clean']  = test_df['statement'].apply(clean_text)

In [10]:
MAX_VOCAB = 20000
MAX_LEN = 50

tok = Tokenizer(num_words=MAX_VOCAB, oov_token='<OOV>')
tok.fit_on_texts(train_df['clean'])

X_train = pad_sequences(tok.texts_to_sequences(train_df['clean']), maxlen=MAX_LEN, padding='post')
X_val   = pad_sequences(tok.texts_to_sequences(val_df['clean']),   maxlen=MAX_LEN, padding='post')
X_test  = pad_sequences(tok.texts_to_sequences(test_df['clean']),  maxlen=MAX_LEN, padding='post')

y_train = tf.keras.utils.to_categorical(train_df['label_enc'], num_labels)
y_val   = tf.keras.utils.to_categorical(val_df['label_enc'],   num_labels)
y_test  = tf.keras.utils.to_categorical(test_df['label_enc'],  num_labels)

In [12]:
bilstm_model = Sequential([
    Embedding(MAX_VOCAB, 128, input_length=MAX_LEN),
    Bidirectional(LSTM(64)),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dense(num_labels, activation='softmax')
])

bilstm_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
early_stop = EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)
bilstm_model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=10,
    batch_size=32,
    callbacks=[early_stop]
)

Epoch 1/10
320/320 ━━━━━━━━━━━━━━━━━━━━ 7s 11ms/step - accuracy: 0.1972 - loss: 1.7594 - val_accuracy: 0.2430 - val_loss: 1.7087
Epoch 2/10
320/320 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.2868 - loss: 1.6393 - val_accuracy: 0.2321 - val_loss: 1.7506
Epoch 3/10
320/320 ━━━━━━━━━━━━━━━━━━━━ 6s 13ms/step - accuracy: 0.4466 - loss: 1.3931 - val_accuracy: 0.2313 - val_loss: 1.9169


In [13]:
print(bilstm_model.evaluate(X_test, y_test))
def predict_bilstm(text):
    seq = pad_sequences(tok.texts_to_sequences([clean_text(text)]), maxlen=MAX_LEN, padding='post')
    pred = bilstm_model.predict(seq)
    return encoder.inverse_transform([np.argmax(pred)])

40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.2292 - loss: 1.7300
[1.7126680612564087, 0.23756906390190125]


In [14]:
full = DatasetDict({
    'train': Dataset.from_pandas(train_df[['statement', 'label']]),
    'validation': Dataset.from_pandas(val_df[['statement', 'label']]),
    'test': Dataset.from_pandas(test_df[['statement', 'label']])
})

bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
def tokenize_fn(batch):
    enc = bert_tokenizer(batch['statement'], padding='max_length', truncation=True, max_length=128)
    enc['labels'] = encoder.transform(batch['label'])
    return enc

raw = full.map(tokenize_fn, batched=True)
raw.set_format(type='tensorflow', columns=['input_ids','attention_mask','labels'])

tf_train = raw['train'].to_tf_dataset(columns=['input_ids','attention_mask'], label_cols=['labels'], shuffle=True, batch_size=8)

tf_val   = raw['validation'].to_tf_dataset(columns=['input_ids','attention_mask'], label_cols=['labels'], shuffle=False, batch_size=8)

tf_test  = raw['test'].to_tf_dataset(columns=['input_ids','attention_mask'], label_cols=['labels'], shuffle=False, batch_size=8)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Map:   0%|          | 0/10240 [00:00<?, ? examples/s]

Map:   0%|          | 0/1284 [00:00<?, ? examples/s]

Map:   0%|          | 0/1267 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/datasets/arrow_dataset.py:400: FutureWarning: The output of `to_tf_dataset` will change when a passing single element list for `labels` or `columns` in the next datasets version. To return a tuple structure rather than dict, pass a single string.
Old behaviour: columns=['a'], labels=['labels'] -> (tf.Tensor, tf.Tensor)  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor)  
New behaviour: columns=['a'],labels=['labels'] -> ({'a': tf.Tensor}, {'labels': tf.Tensor})  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor) 
  warnings.warn(


In [15]:
from tensorflow.keras.optimizers import Adam

In [16]:
bert_model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=num_labels)
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
bert_model.compile(optimizer='adam',
                   loss=loss_fn,
                   metrics=['accuracy'])
bert_model.optimizer.learning_rate.assign(2e-5)

bert_model.fit(
    tf_train,
    validation_data=tf_val,
    epochs=10
)


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/10
1280/1280 [==============================] - 346s 237ms/step - loss: 1.7247 - accuracy: 0.2363 - val_loss: 1.6837 - val_accuracy: 0.2586
Epoch 2/10
1280/1280 [==============================] - 300s 235ms/step - loss: 1.6298 - accuracy: 0.2975 - val_loss: 1.6581 - val_accuracy: 0.2773
Epoch 3/10
1280/1280 [==============================] - 308s 241ms/step - loss: 1.3838 - accuracy: 0.4358 - val_loss: 1.7658 - val_accuracy: 0.2562
Epoch 4/10
1280/1280 [==============================] - 308s 241ms/step - loss: 0.9276 - accuracy: 0.6369 - val_loss: 2.2241 - val_accuracy: 0.2617
Epoch 5/10
1280/1280 [==============================] - 301s 235ms/step - loss: 0.5214 - accuracy: 0.8063 - val_loss: 2.7892 - val_accuracy: 0.2749
Epoch 6/10
1280/1280 [==============================] - 300s 235ms/step - loss: 0.2816 - accuracy: 0.9031 - val_loss: 3.2978 - val_accuracy: 0.2477
Epoch 7/10
1280/1280 [==============================] - 308s 240ms/step - loss: 0.1996 - accuracy: 0.9286 - val_

In [23]:
print(bert_model.evaluate(tf_test))
def predict_bert(text):
    enc = bert_tokenizer(text, return_tensors='tf', truncation=True, padding='max_length', max_length=128)
    logits = bert_model(enc).logits
    return encoder.inverse_transform([tf.argmax(logits, axis=1).numpy()[0]])

print(predict_bilstm("Alex Sink funneled three quarters of a million dollars in no-bid contracts to Bank of America."))
print(predict_bert("Transgender individuals in the U.S. have a 1-in-12 chance of being murdered."))


159/159 [==============================] - 13s 80ms/step - loss: 4.0886 - accuracy: 0.2526
[4.088626861572266, 0.2525651156902313]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
['half-true']
['pants-fire']
